# Neural Transfer
### Import Modules

In [4]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

print("modules loaded...\nversion of Pytorch -> {}".format(torch.__version__))

modules loaded
 version of Pytorch -> 1.2.0


### Check Python Version

In [3]:
from platform import python_version
print("version of python used -> {}".format(python_version()))

version of python used -> 3.7.4
